<a href="https://colab.research.google.com/github/kumarseshu123/VCC/blob/main/filter_pruning_based_on_importance_and_redundancy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from tensorflow.keras import datasets,models,layers
from scipy.stats import pearsonr
from sklearn.metrics.cluster import adjusted_mutual_info_score


####################################################################
################   HELPER FUNCTIONS.  ##############################
####################################################################


#Calculate the entopy of the filter outputs
def filter_average_pooled_entropy(average_pooled_filter_output):
  # Find the number of classes in this distribution 
  num_channels   = average_pooled_filter_output.shape[1]
  num_examples = average_pooled_filter_output.shape[0]
  
  entropy_channel = np.zeros(num_channels)

  # Measure entropy per channel 
  for channel_idx in range(0, num_channels, 1):    
    hist_f1, bins_f1 = np.histogram(average_pooled_filter_output[ :,channel_idx ], bins=10)
    classes_f1       = np.digitize(average_pooled_filter_output[ :,channel_idx], bins_f1)
    bin_count        = np.bincount(classes_f1) + 1 
    bin_probablities = bin_count/(num_examples)
      
    # Find the log of each probablity 
    bin_probablities = bin_probablities  
    log_bin_probablities = np.log(bin_probablities)

    # Shannon probablity is  Sum (-( p(x)) * log (p(x)))
    bin_probablities_product = -1 * np.multiply(log_bin_probablities, bin_probablities) 
    
    #Sum across all classes 
    entropy_channel[channel_idx] = np.sum(bin_probablities_product)
    
  return (entropy_channel)


# Calculate the variance of the output per batch
def batch_variance(outputs):
  return (tf.math.reduce_variance(outputs, axis=0))

####################################################################
##########   BASE MODEL    #########################################
####################################################################


# Load the CIFAR10 data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize the data
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

# Convert the labels to categorical data
y_train_cat = keras.utils.to_categorical(y_train, 10)
y_test_cat = keras.utils.to_categorical(y_test, 10)

# Define a VGG model
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(32, 32, 3)))
model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train_cat, epochs=25, batch_size=128)
print(model.summary())

# Test the accuracy of the base model 
ModelLoss, ModelAccuracy = model.evaluate(x_train, y_train_cat)

print('Model Accuracy BASE is {}'.format(ModelAccuracy))


####################################################################
######### PART 1 - MEASURE ENTRORY OF THE FILTER AND PRUNE #########
####################################################################

# Create an extactor model 
extractor = keras.Model(inputs=model.inputs, outputs=[layer.output for layer in model.layers])

# Create a numpy array with dimentions (num_example, num_hidden_layer_channels)
hidden_layer_average_pool = np.zeros((len(x_train),64), dtype=float)
batch_output_entropy = []

# For the entire training data, we need to find the filter outputs 
# Create a list per filter (2-D numpy array) of the average values of the filter outputs 
# For this version, we are only pruning the layer 4 
for batch_idx in range(0, (len(x_train)), 25):
  # Measure the mutual information between each filter   
  first_conv_layer_output = extractor(x_train[batch_idx:batch_idx+25])
  first_conv_layer_output_average_pool = tf.reduce_mean(first_conv_layer_output[4], axis=[1, 2])
  hidden_layer_average_pool[batch_idx:batch_idx+25] = first_conv_layer_output_average_pool

# Measure the entropy of the channel
hidden_layer_average_pool_entropy = filter_average_pooled_entropy(hidden_layer_average_pool)

filters_pruned = []
filters_already_processed = []

# Remove all nodes that have low entropy s
for hidden_node in range(0, hidden_layer_average_pool.shape[1]):
  if( hidden_layer_average_pool_entropy[hidden_node] < 0.4):
      print("Setting weight to zero for filter", hidden_node)
      
      layer = model.layers[4]
      W_matrix = layer.weights[0]
      b_list = layer.weights[1]

      # Set the weights of the W matrix as 0 
      mask_b = np.ones(b_list.shape)
      mask_b[hidden_node] = 0
      mask_b_tensor  = tf.convert_to_tensor(mask_b, dtype=float)
      new_b_list     = tf.math.multiply(mask_b_tensor, b_list)

      # Set the bias term as 0
      mask = np.ones(W_matrix.shape)
      mask[:,:,:,hidden_node] = 0
      mask_tensor  = tf.convert_to_tensor(mask, dtype=float)
      new_W_matrix = tf.math.multiply(mask_tensor, W_matrix)

      # Assign new weights to the model 
      new_weights = [new_W_matrix, new_b_list]
      model.layers[4].set_weights(new_weights)

      filters_already_processed.append(hidden_node)
      filters_pruned.append(hidden_node)

ModelLoss, ModelAccuracy = model.evaluate(x_train, y_train_cat)

print('Model Accuracy AFTER ENTROPY  is {}'.format(ModelAccuracy))



####################################################################
###### PART 2 - PERFORM SIMILARITY MAPPING AND PRUNE ###############
####################################################################

# Measure the mutual information between all the filters 
similarity_mapping = np.zeros((hidden_layer_average_pool.shape[1], hidden_layer_average_pool.shape[1]), dtype=float)


for f1_idx in range(0, (hidden_layer_average_pool.shape[1]), 1):
  for f2_idx in range(f1_idx, (hidden_layer_average_pool.shape[1]), 1):  
    corr, p_value = pearsonr(hidden_layer_average_pool[ :,f2_idx ], hidden_layer_average_pool[ :,f1_idx ])
    similarity_mapping[f1_idx, f2_idx] = abs(corr)


for pruning_index in range(0, 10, 1):  
  max_similarity = 0 ;
  max_similarity_filters_index  = []
  for f1_idx in range(0, (hidden_layer_average_pool.shape[1]), 1):
    for f2_idx in range(f1_idx, (hidden_layer_average_pool.shape[1]), 1):
      if (similarity_mapping[f1_idx, f2_idx] > max_similarity):
        if(f1_idx != f2_idx):
          if ((f1_idx not in filters_already_processed) and (f2_idx not in filters_already_processed)):
            max_similarity =  similarity_mapping[f1_idx, f2_idx]
            max_similarity_filters_index.clear()
            max_similarity_filters_index.append(f1_idx)
            max_similarity_filters_index.append(f2_idx)
 
  if(max_similarity != 0) : 
    filters_already_processed.append(max_similarity_filters_index[0])
    filters_already_processed.append(max_similarity_filters_index[1])
    filters_pruned.append(max_similarity_filters_index[0])

# Prune away the one of the filters that are similar 
for hidden_node in range(0, hidden_layer_average_pool.shape[1]):
  if( hidden_node in  filters_pruned):
      print("Setting weight to zero for filter", hidden_node)
      layer = model.layers[4]
      W_matrix = layer.weights[0]
      b_list = layer.weights[1]

      # Set the weights of the W matrix as 0 
      mask_b = np.ones(b_list.shape)
      mask_b[hidden_node] = 0
      mask_b_tensor  = tf.convert_to_tensor(mask_b, dtype=float)
      new_b_list     = tf.math.multiply(mask_b_tensor, b_list)

      # Set the bias term as 0
      mask = np.ones(W_matrix.shape)
      mask[:,:,:,hidden_node] = 0
      mask_tensor  = tf.convert_to_tensor(mask, dtype=float)
      new_W_matrix = tf.math.multiply(mask_tensor, W_matrix)

      # Assign new weights to the model 
      new_weights = [new_W_matrix, new_b_list]
      model.layers[4].set_weights(new_weights)


####################################################################
################  COLLECT ACCURACY AFTER PRUNING  ################## 
####################################################################

ModelLoss, ModelAccuracy = model.evaluate(x_train, y_train_cat)
print('Model Accuracy AFTER SIMILARIY is {}'.format(ModelAccuracy))
model.fit(x_train, y_train_cat, epochs=3, batch_size=128)
ModelLoss, ModelAccuracy = model.evaluate(x_train, y_train_cat)

print('Model Loss is {}'.format(ModelLoss))
print('Model Accuracy is {}'.format(ModelAccuracy))
print('Print model size and other info')
print(model.summary())

Epoch 1/25
391/391 [==============================] - 7s 10ms/step - loss: 1.4790 - accuracy: 0.4644
Epoch 2/25
391/391 [==============================] - 4s 10ms/step - loss: 1.0050 - accuracy: 0.6479
Epoch 3/25
391/391 [==============================] - 4s 11ms/step - loss: 0.8179 - accuracy: 0.7144
Epoch 4/25
391/391 [==============================] - 4s 10ms/step - loss: 0.7032 - accuracy: 0.7542
Epoch 5/25
391/391 [==============================] - 4s 10ms/step - loss: 0.6084 - accuracy: 0.7887
Epoch 6/25
391/391 [==============================] - 4s 11ms/step - loss: 0.5280 - accuracy: 0.8150
Epoch 7/25
391/391 [==============================] - 4s 10ms/step - loss: 0.4582 - accuracy: 0.8397
Epoch 8/25
391/391 [==============================] - 4s 11ms/step - loss: 0.3789 - accuracy: 0.8696
Epoch 9/25
391/391 [==============================] - 5s 12ms/step - loss: 0.3076 - accuracy: 0.8932
Epoch 10/25
391/391 [==============================] - 4s 10ms/step - loss: 0.2508 - accura